In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
readIn = pd.read_csv("data/train_df.csv")
train_df = pd.DataFrame(readIn)

In [3]:
readIn = pd.read_csv("data/res_df.csv")
df = pd.DataFrame(readIn)

In [4]:
readIn = pd.read_csv("data/data_submission_example.csv")
df_target = pd.DataFrame(readIn)
# df_target.head(20)

# print(df_target.shape)
# print(df_target['BORROWER_ID'].unique)

In [5]:
df

,BORROWER_ID,target,CONTRACT_ID_SUM,CONTRACT_CREDIT_LOSS_SUM,CONTRACT_DEPT_SERVICE_TO_INCOME_MEAN,CONTRACT_INCOME_SUM,CONTRACT_INSTALMENT_AMOUNT_MEAN,CONTRACT_INTEREST_RATE_MEAN,CONTRACT_LGD_SUM,CONTRACT_LOAN_AMOUNT_SUM,...,CONTRACT_REFINANCED_10001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_0.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100003.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100004.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_110001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140003.0
0,d8SqtuEV,0,3,13831.0,27.76,256244.0,27149.0,17.31,3295461.0,3367556,...,0,2,0,1,0,0,0,0,0,0
1,lrdxML0g,0,1,0.0,0.00,0.0,9528.0,16.57,994868.0,55590,...,0,1,0,0,0,0,0,0,0,0
2,gII7nnq4,0,1,16350.0,7.05,127305.0,8899.0,22.35,282553.0,275924,...,0,0,0,0,1,0,0,0,0,0
3,MMkJ8z/e,0,1,0.0,0.00,0.0,14329.0,0.00,68981.0,138862,...,0,1,0,0,0,0,0,0,0,0
4,M417onFP,0,1,2395.0,0.00,0.0,4003.0,23.92,163319.0,169624,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149846,POeQToeq,0,1,0.0,0.00,0.0,0.0,0.00,0.0,211813,...,0,1,0,0,0,0,0,0,0,0
1149847,C0amt32x,0,1,0.0,0.00,0.0,0.0,0.00,0.0,485818,...,0,1,0,0,0,0,0,0,0,0
1149848,WW5vgE+y,0,1,0.0,0.00,0.0,0.0,0.00,0.0,5866242,...,0,1,0,0,0,0,0,0,0,0
1149849,26ZlUKOa,0,1,0.0,0.00,0.0,0.0,0.00,0.0,80914,...,0,1,0,0,0,0,0,0,0,0


In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Bemenő változók és célváltozó meghatározása
bemenő_változók = df.columns.drop(['BORROWER_ID', 'target'])  # Az összes oszlop, kivéve a 'BORROWER_ID' és 'target'
X = train_df[bemenő_változók]  # Bemenő változók
y = train_df['target']  # Célváltozó



In [7]:
# Modell inicializálása és illesztése az adatokra
gbm = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=22)
gbm.fit(X, y)

GradientBoostingClassifier(n_estimators=50, random_state=22)

# SAVING RESULTS

In [8]:
merged_df = df[df['BORROWER_ID'].isin(df_target['BORROWER_ID'])]

In [9]:
X = merged_df[bemenő_változók]

In [10]:
len(X)

1117674

In [11]:
import joblib

# joblib.dump(gbm, 'gradient_boosting_model.pkl')

In [12]:
# gbm = joblib.load('gradient_boosting_model.pkl')

In [13]:
merged_df['predicted_target'] = gbm.predict_proba(X)[:,1]

C:\Users\User\AppData\Local\Temp\ipykernel_13420\260587743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['predicted_target'] = gbm.predict_proba(X)[:,1]


In [14]:
# merged_df2=pd.merge(df_target,merged_df, on='BORROWER_ID', how='left')

In [15]:
# cols=['BORROWER_ID', 'predicted_target', 'PRED']

# res=merged_df2[cols]

NameError: name 'merged_df2' is not defined

In [16]:
merged_df

,BORROWER_ID,target,CONTRACT_ID_SUM,CONTRACT_CREDIT_LOSS_SUM,CONTRACT_DEPT_SERVICE_TO_INCOME_MEAN,CONTRACT_INCOME_SUM,CONTRACT_INSTALMENT_AMOUNT_MEAN,CONTRACT_INTEREST_RATE_MEAN,CONTRACT_LGD_SUM,CONTRACT_LOAN_AMOUNT_SUM,...,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_0.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100003.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100004.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_110001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140003.0,predicted_target
0,d8SqtuEV,0,3,13831.0,27.76,256244.0,27149.0,17.31,3295461.0,3367556,...,2,0,1,0,0,0,0,0,0,0.00066
1,lrdxML0g,0,1,0.0,0.00,0.0,9528.0,16.57,994868.0,55590,...,1,0,0,0,0,0,0,0,0,0.00067
2,gII7nnq4,0,1,16350.0,7.05,127305.0,8899.0,22.35,282553.0,275924,...,0,0,0,1,0,0,0,0,0,0.00067
3,MMkJ8z/e,0,1,0.0,0.00,0.0,14329.0,0.00,68981.0,138862,...,1,0,0,0,0,0,0,0,0,0.00067
4,M417onFP,0,1,2395.0,0.00,0.0,4003.0,23.92,163319.0,169624,...,0,0,1,0,0,0,0,0,0,0.00067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149846,POeQToeq,0,1,0.0,0.00,0.0,0.0,0.00,0.0,211813,...,1,0,0,0,0,0,0,0,0,0.00067
1149847,C0amt32x,0,1,0.0,0.00,0.0,0.0,0.00,0.0,485818,...,1,0,0,0,0,0,0,0,0,0.00067
1149848,WW5vgE+y,0,1,0.0,0.00,0.0,0.0,0.00,0.0,5866242,...,1,0,0,0,0,0,0,0,0,0.00067
1149849,26ZlUKOa,0,1,0.0,0.00,0.0,0.0,0.00,0.0,80914,...,1,0,0,0,0,0,0,0,0,0.00067


In [17]:
cols=['BORROWER_ID', 'predicted_target']

res=merged_df[cols]

In [18]:
res['predicted_target'].isna().sum()

0

In [19]:
res.head(10)


,BORROWER_ID,predicted_target
0,d8SqtuEV,0.00066
1,lrdxML0g,0.00067
2,gII7nnq4,0.00067
3,MMkJ8z/e,0.00067
4,M417onFP,0.00067
5,u9GRAuen,0.00067
6,XPWFcJZI,0.00066
7,L2Nu2zZe,0.00066
8,5A0GWOi2,0.00066
9,lqPGd2+x,0.00067


In [20]:
res.columns=['BORROWER_ID', 'PRED']

In [21]:
# multi = 0.0148/res['PRED'].mean()
# res['PRED']=res['PRED']*multi
res['PRED'].mean()

0.008629810131342704

In [22]:
print(res.shape)
res.to_csv('data/results.csv', index=False)

(1117674, 2)


In [ ]:
df_target.shape

In [ ]:
res['PRED'].mean()